<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/faiss/faiss_Clustering_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kstathou/vector_engine

Cloning into 'vector_engine'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 74 (delta 32), reused 59 (delta 18), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [2]:
cd vector_engine

/content/vector_engine


In [3]:
pip install -r requirements.txt

Obtaining file:///content/vector_engine (from -r requirements.txt (line 9))
     |████████████████████████████████| 804.1 MB 2.9 kB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 66 kB 5.3 MB/s 
     |████████████████████████████████| 10.5 MB 48.1 MB/s 
     |████████████████████████████████| 7.1 MB 31.1 MB/s 
     |████████████████████████████████| 14.5 MB 50.1 MB/s 
     |████████████████████████████████| 69 kB 7.6 MB/s 
     |████████████████████████████████| 7.1 MB 28.0 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 3.0 MB 42.2 MB/s 
     |████████████████████████████████| 895 kB 40.6 MB/s 
     |████████████████████████████████| 131 kB 49.7 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 111 kB 49.9 MB/s 
     |████████████████████████████████| 8.5 MB 46.2 MB/s 
     |████████████████████████████████| 4.3 MB 43.1 MB/s 

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2
import pandas as pd

import torch
from sentence_transformers import SentenceTransformer

import faiss
import numpy as np
import pickle
from pathlib import Path

from vector_engine.utils import vector_search, id2details

In [6]:
df = pd.read_csv('data/misinformation_papers.csv')

In [7]:
df.head(3)

,original_title,abstract,year,citations,id,is_EN
0,When Corrections Fail: The Persistence of Poli...,An extensive literature addresses citizen igno...,2010,901,2132553681,1
1,A postmodern Pandora's box: anti-vaccination m...,The Internet plays a large role in disseminati...,2010,440,2117485795,1
2,Spread of (Mis)Information in Social Networks,We provide a model to investigate the tension ...,2010,278,2120015072,1


In [8]:
print(f"Misinformation, disinformation and fake news papers: {df.id.unique().shape[0]}")

Misinformation, disinformation and fake news papers: 8430


distilbert-base-nli-stsb-mean-tokens 소환! 
버트보다는 약간 성능이 낮지만 훨씬 작은 사이즈!

In [9]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

100%|██████████| 245M/245M [00:12<00:00, 20.2MB/s]


In [10]:
if torch.cuda.is_available():
  model = model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [11]:
embeddings = model.encode(df.abstract.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/264 [00:00<?, ?it/s]

In [12]:
print('shape of vectorized abstract', {embeddings[0].shape})

shape of vectorized abstract {(768,)}


In [14]:
# 1. 데이터 타입 변경
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# 2. 인덱스 초기화
index = faiss.IndexFlatL2(embeddings.shape[1])

# 3. 인덱스를 IndexIDMap으로 전달
index = faiss.IndexIDMap(index)

# 4. 고유 Id와 vector을 추가
index.add_with_ids(embeddings, df.id.values)

print(f'faiss index에 있는 벡터의 수: {index.ntotal}')

faiss index에 있는 벡터의 수: 8430


In [15]:
df.iloc[5415,1]

"We address the diffusion of information about the COVID-19 with a massive data analysis on Twitter, Instagram, YouTube, Reddit and Gab. We analyze engagement and interest in the COVID-19 topic and provide a differential assessment on the evolution of the discourse on a global scale for each platform and their users. We fit information spreading with epidemic models characterizing the basic reproduction number [Formula: see text] for each social media platform. Moreover, we identify information spreading from questionable sources, finding different volumes of misinformation in each platform. However, information from both reliable and questionable sources do not present different spreading patterns. Finally, we provide platform-dependent numerical estimates of rumors' amplification."

In [17]:
#10개의 가까운 이웃을 추출
D, I = index.search(np.array([embeddings[5415]]), k=10)
print(f'L2 거리 {D.flatten().tolist()} \n\nMAG paper IDs: {I.flatten().tolist()}')

L2 거리 [0.0, 1.2672882080078125, 62.72166442871094, 63.670326232910156, 64.58393859863281, 67.47343444824219, 67.96401977539062, 69.47561645507812, 72.56331634521484, 74.62234497070312] 

MAG paper IDs: [3092618151, 3011345566, 3012936764, 3055557295, 3011186656, 3044429417, 3092128270, 3024620668, 3047284882, 3048848247]


In [18]:
# index에 기반한 title 추출
id2details(df, I, 'original_title')

[['The COVID-19 social media infodemic.'],
 ['The COVID-19 Social Media Infodemic'],
 ['Understanding the perception of COVID-19 policies by mining a multilanguage Twitter dataset'],
 ['Covid-19 infodemic reveals new tipping point epidemiology and a revised R formula.'],
 ['Coronavirus Goes Viral: Quantifying the COVID-19 Misinformation Epidemic on Twitter'],
 ['Effects of misinformation on COVID-19 individual responses and recommendations for resilience of disastrous consequences of misinformation'],
 ['Analysis of online misinformation during the peak of the COVID-19 pandemics in Italy'],
 ['Quantifying COVID-19 Content in the Online Health Opinion War Using Machine Learning'],
 ['Global Infodemiology of COVID-19: Analysis of Google Web Searches and Instagram Hashtags.'],
 ['COVID-19-Related Infodemic and Its Impact on Public Health: A Global Social Media Analysis.']]

In [19]:
# index에 기반한 초록 추출
id2details(df, I, 'abstract')

[["We address the diffusion of information about the COVID-19 with a massive data analysis on Twitter, Instagram, YouTube, Reddit and Gab. We analyze engagement and interest in the COVID-19 topic and provide a differential assessment on the evolution of the discourse on a global scale for each platform and their users. We fit information spreading with epidemic models characterizing the basic reproduction number [Formula: see text] for each social media platform. Moreover, we identify information spreading from questionable sources, finding different volumes of misinformation in each platform. However, information from both reliable and questionable sources do not present different spreading patterns. Finally, we provide platform-dependent numerical estimates of rumors' amplification."],
 ["We address the diffusion of information about the COVID-19 with a massive data analysis on Twitter, Instagram, YouTube, Reddit and Gab. We analyze engagement and interest in the COVID-19 topic and p

In [20]:
user_query = """
WhatsApp was alleged to have been widely used to spread misinformation and propaganda 
during the 2018 elections in Brazil and the 2019 elections in India. Due to the 
private encrypted nature of the messages on WhatsApp, it is hard to track the dissemination 
of misinformation at scale. In this work, using public WhatsApp data from Brazil and India, we 
observe that misinformation has been largely shared on WhatsApp public groups even after they 
were already fact-checked by popular fact-checking agencies. This represents a significant portion 
of misinformation spread in both Brazil and India in the groups analyzed. We posit that such 
misinformation content could be prevented if WhatsApp had a means to flag already fact-checked 
content. To this end, we propose an architecture that could be implemented by WhatsApp to counter 
such misinformation. Our proposal respects the current end-to-end encryption architecture on WhatsApp, 
thus protecting users’ privacy while providing an approach to detect the misinformation that benefits 
from fact-checking efforts.
"""

In [21]:
# 벡터 serach function에 전부 담았다.
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [7.38446044921875, 57.32252502441406, 57.32252502441406, 71.48451232910156, 72.06806945800781, 79.134765625, 86.0127944946289, 89.91023254394531, 90.76019287109375, 90.76422119140625]

MAG paper IDs: [3047438096, 3021927925, 3037966274, 2889959140, 2791045616, 2943077655, 3014380170, 2967434249, 3028584171, 2990343632]


In [22]:
# 인덱스에 기반한 타이틀 추출
id2details(df, I, 'original_title')

[['Can WhatsApp Benefit from Debunked Fact-Checked Stories to Reduce Misinformation?'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A Dataset of Fact-Checked Images Shared on WhatsApp During the Brazilian and Indian Elections'],
 ['A System for Monitoring Public Political Groups in WhatsApp'],
 ['Politics of Fake News: How WhatsApp Became a Potent Propaganda Tool in India'],
 ['Characterizing Attention Cascades in WhatsApp Groups'],
 ['OS IMPACTOS JURÍDICOS E SOCIAIS DAS FAKE NEWS EM TERRITÓRIO BRASILEIRO'],
 ['Fake News and Social Media: Indian Perspective'],
 ['Images and Misinformation in Political Groups: Evidence from WhatsApp in India'],
 ['Can WhatsApp Counter Misinformation by Limiting Message Forwarding']]

In [23]:
# 구글 코랩에서 돌리면 인덱스를 0으로 수정해야함, 로컬에서는 1
project_dir = Path('notebooks').resolve().parents[0]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/content/vector_engine
